In [2]:
from util import local_settings
from openai import OpenAI

print("First LLM API example")
print(f"✅ OpenAI Key loaded ({local_settings.OPENAI_API_KEY[0:-15]}...)")

client = OpenAI(api_key=local_settings.OPENAI_API_KEY)

sk-Ez70Zdl8M6eOe50Uu68DT3BlbkFJgDj6B
First LLM API example
✅ OpenAI Key loaded (sk-Ez70Zdl8M6eOe50Uu68DT3BlbkFJgDj6B...)


In [3]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0, messages=None):
    if not messages:
        messages = [{"role": "user", "content": prompt}]
    else:
        messages.append({"role": "user", "content": prompt})

    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

    return completion.choices[0].message.content

In [6]:
messages = [
    {
        'role': 'system',
        'content': """
            You are ReviewBot, an automated service to generate reviews for places to visit in Lisbon. 
            By pretending ypu are a person that has visited the place, \
                generate a review based on the provided information about the place. 
            The length of the review must be between 10 and 100 words.

            The dataset includes information about places, their features, a rating and the number of ratings per star.

            Example dataset row:
            Place: Belém Tower
            Features: Historical landmark
            Rating: 4.5/5
        """
    }
]

In [7]:
prompt = "Belém Tower"

response = get_completion(prompt, messages=messages)
print(response)

messages.append({'role':'assistant', 'content':f"{response}"})

I recently had the pleasure of visiting Belém Tower in Lisbon and I must say, it was an incredible experience. This historical landmark truly captures the essence of Lisbon's rich history and culture.

The tower itself is a magnificent structure, standing tall and proud by the Tagus River. Its unique architecture and intricate details are a testament to the craftsmanship of the past. As I walked through the various rooms and corridors, I couldn't help but feel a sense of awe and wonder at the history that unfolded within these walls.

The views from the top of the tower are simply breathtaking. From there, I could see the vast expanse of the river, the city skyline, and even the iconic 25 de Abril Bridge in the distance. It was a perfect spot to take in the beauty of Lisbon and capture some memorable photos.

The staff at Belém Tower were friendly and knowledgeable, providing interesting insights into the tower's history and significance. They were more than happy to answer any questio

In [69]:
import pandas as pd

# Load the CSV file into a DataFrame
trip_df = pd.read_csv('LisbonTrip_Final.csv')

# Extract a value from the 'Link' column to create a new 'Ranking' column
trip_df['Ranking'] = trip_df['Link'].str.split('.').str[0]

# Convert the 'Ranking' column to numeric values
trip_df['Ranking'] = pd.to_numeric(trip_df['Ranking'], errors='coerce')

# Drop unnecessary columns
trip_df.drop(['web-scraper-order', 'web-scraper-start-url', 'Link'], axis=1, inplace=True)

# Set the 'Ranking' column as the index
trip_df.set_index('Ranking', inplace=True)

# Sort the DataFrame based on the index
trip_df.sort_index(inplace=True)

# Display the modified DataFrame
trip_df


,Link-href,Name,Types,About,Schedule,Duration,Address,Rating,ReviewsNo,ExcellentRating,VeryGoodRating,AverageRating,PoorRating,TerribleRating
Ranking,,,,,,,,,,,,,,
1,https://www.tripadvisor.com/Attraction_Review-...,Oceanário de Lisboa,"[{""Types"":""Aquariums""}]",Oceanário de Lisboa is one of the largest aqua...,10:00 AM - 8:00 PM,Duration: 2-3 hours,"Esplanada D. Carlos I Doca dos Olivais, Lisbon...",4.5,"40,565 reviews","26,272","10,703","2,897",487,NaN
2,https://www.tripadvisor.com/Attraction_Review-...,Quinta da Regaleira,"[{""Types"":""Architectural Buildings""},{""Types"":...",This fairytale-like country estate and UNESCO ...,10:00 AM - 5:30 PM,Duration: 1-2 hours,"Rua Barbosa do Bocage, Sintra 2710-567 Portugal",5.0,"13,871 reviews","11,321","2,063",337,81,69.0
3,https://www.tripadvisor.com/Attraction_Review-...,Alfama,"[{""Types"":""Neighborhoods""}]",Wonderful sights delight walkers at every turn...,NaN,Duration: 1-2 hours,NaN,4.5,"21,287 reviews","13,203","6,725","1,081",178,100.0
4,https://www.tripadvisor.com/Attraction_Review-...,Mosteiro dos Jeronimos,"[{""Types"":""Historic Sites""},{""Types"":""Architec...",No expense was spared when they built this mas...,10:00 AM - 6:00 PM,Duration: 2-3 hours,"Praca do Imperio Belém, Lisbon 1400-206 Portugal",4.5,"31,687 reviews","19,522","9,073","2,354",436,NaN
5,https://www.tripadvisor.com/Attraction_Review-...,Torre de Belém,"[{""Types"":""Points of Interest & Landmarks""}]",A 16th-century tower which serves as a monumen...,10:00 AM - 5:00 PM,Duration: 2-3 hours,"Avenida Brasília - Belém, Lisbon 1400-038 Port...",4.0,"37,917 reviews","17,117","14,123","5,193",978,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4646,https://www.tripadvisor.com/Attraction_Review-...,Taxi Boat Lisboa,[],Taxi rides and excursions around Lisbon and Ta...,12:00 AM - 11:59 PM,Duration: 1-2 hours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4647,https://www.tripadvisor.com/Attraction_Review-...,Sailboat Aqui Há Peixe,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4648,https://www.tripadvisor.com/Attraction_Review-...,Acervo,[],Founded in March 2016 and based in the former ...,NaN,Duration: < 1 hour,"Rua do Machadinho 1, Lisbon 1350-003 Portugal",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
# Function to extract only the time part
def extract_times(time_range):
    if time_range == 'Closed until further notice':
        return pd.NaT, pd.NaT, pd.NaT, pd.NaT
    elif pd.notna(time_range):
        # Split the string into two parts
        parts = time_range.split(' - ')
        
        # Check if there are exactly two parts
        if len(parts) == 2:
            return pd.to_datetime(parts[0]).time(), pd.to_datetime(parts[1]).time(), pd.NaT, pd.NaT
        else:
            # Has lunch break
            lunch_time = parts[1].split(', ')
            return pd.to_datetime(parts[0]).time(), pd.to_datetime(parts[2]).time(), pd.to_datetime(lunch_time[0]).time(), pd.to_datetime(lunch_time[1]).time()
    else:
        return pd.NaT, pd.NaT, pd.NaT, pd.NaT

# Apply the function to create four new columns: 'StartTime', 'EndTime', 'LunchStart', and 'LunchEnd'
time_columns = trip_df['Schedule'].apply(extract_times).apply(pd.Series)
trip_df.loc[:, ['StartTime', 'EndTime', 'LunchStart', 'LunchEnd']] = time_columns

# Drop rows of closed activitiess
trip_df = trip_df[trip_df['Schedule'] != 'Closed until further notice']

trip_df


,Link-href,Name,Types,About,Schedule,Duration,Address,Rating,ReviewsNo,ExcellentRating,VeryGoodRating,AverageRating,PoorRating,TerribleRating,StartTime,EndTime,LunchStart,LunchEnd
Ranking,,,,,,,,,,,,,,,,,,
1,https://www.tripadvisor.com/Attraction_Review-...,Oceanário de Lisboa,"[{""Types"":""Aquariums""}]",Oceanário de Lisboa is one of the largest aqua...,10:00 AM - 8:00 PM,Duration: 2-3 hours,"Esplanada D. Carlos I Doca dos Olivais, Lisbon...",4.5,"40,565 reviews","26,272","10,703","2,897",487,NaN,NaN,NaN,NaN,NaN
2,https://www.tripadvisor.com/Attraction_Review-...,Quinta da Regaleira,"[{""Types"":""Architectural Buildings""},{""Types"":...",This fairytale-like country estate and UNESCO ...,10:00 AM - 5:30 PM,Duration: 1-2 hours,"Rua Barbosa do Bocage, Sintra 2710-567 Portugal",5.0,"13,871 reviews","11,321","2,063",337,81,69.0,NaN,NaN,NaN,NaN
3,https://www.tripadvisor.com/Attraction_Review-...,Alfama,"[{""Types"":""Neighborhoods""}]",Wonderful sights delight walkers at every turn...,NaN,Duration: 1-2 hours,NaN,4.5,"21,287 reviews","13,203","6,725","1,081",178,100.0,NaN,NaN,NaN,NaN
4,https://www.tripadvisor.com/Attraction_Review-...,Mosteiro dos Jeronimos,"[{""Types"":""Historic Sites""},{""Types"":""Architec...",No expense was spared when they built this mas...,10:00 AM - 6:00 PM,Duration: 2-3 hours,"Praca do Imperio Belém, Lisbon 1400-206 Portugal",4.5,"31,687 reviews","19,522","9,073","2,354",436,NaN,NaN,NaN,NaN,NaN
5,https://www.tripadvisor.com/Attraction_Review-...,Torre de Belém,"[{""Types"":""Points of Interest & Landmarks""}]",A 16th-century tower which serves as a monumen...,10:00 AM - 5:00 PM,Duration: 2-3 hours,"Avenida Brasília - Belém, Lisbon 1400-038 Port...",4.0,"37,917 reviews","17,117","14,123","5,193",978,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4646,https://www.tripadvisor.com/Attraction_Review-...,Taxi Boat Lisboa,[],Taxi rides and excursions around Lisbon and Ta...,12:00 AM - 11:59 PM,Duration: 1-2 hours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4647,https://www.tripadvisor.com/Attraction_Review-...,Sailboat Aqui Há Peixe,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4648,https://www.tripadvisor.com/Attraction_Review-...,Acervo,[],Founded in March 2016 and based in the former ...,NaN,Duration: < 1 hour,"Rua do Machadinho 1, Lisbon 1350-003 Portugal",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
